# Correlation of Netflix's Stock Price and Releases

### Milestone 2: Cleaning Netflix data from Kaggle

In [18]:
# Read the data and check a few rows

import pandas as pd

netflix_df = pd.read_csv("Netflix Dataset Latest 2021.csv")

# Step 1: Drop the 2 "Unnamed" columns at the end of the dataset.
# These columns are being pulled in from the .csv but do not actually contain data

netflix_df = netflix_df.drop('Unnamed: 29', axis=1)
netflix_df = netflix_df.drop('Unnamed: 30', axis=1)

# Step 2: Change "Netflix Release Date" to data type datetime
# This will let us join to other tables based on date and pull out the year in the next step.

netflix_df['Netflix Release Date'] = pd.to_datetime(netflix_df['Netflix Release Date'], errors='coerce')
netflix_df['date'] = netflix_df['Netflix Release Date'].dt.date

# Step 3: Add a "Year" column.
# This column will be used to join to the other data sources

netflix_df['Year'] = netflix_df['Netflix Release Date'].dt.year

# Step 4: Find all NaN or NULL values for the important columns and exclude those rows
# The following three columns are critical to the analysis and values need to be present or the row should be excluded.

# Drop NaN values for "Country Availability"
netflix_df = netflix_df[netflix_df['Country Availability'].notna()]

# Drop NaN values for "Series or Movie"
netflix_df = netflix_df[netflix_df['Series or Movie'].notna()]

# Drop NaN values for "IMDb Score"
netflix_df = netflix_df[netflix_df['IMDb Score'].notna()]

# Step 5: Filter the data frame to only include those titles which have been released in the US.
# Since the analysis is being performed on American stock prices, we are interested only in titles that would impact
# the American stock market.

# Convert column to data type string
netflix_df['Country Availability'] = netflix_df['Country Availability'].astype(str)

# Filter to only include US-released titles and check for row count
netflix_df = netflix_df[netflix_df['Country Availability'].str.contains('United States', case=False, regex=False)]

# Filter to only include necessary columns
netflix_df = netflix_df[['Title', 'Series or Movie', 'Runtime', 'IMDb Score', 'Rotten Tomatoes Score', 
                         'Netflix Release Date']]

In [19]:
# Show the first few rows of the Netflix data set

netflix_df.head(5)

,Title,Series or Movie,Runtime,IMDb Score,Rotten Tomatoes Score,Netflix Release Date
61,The Girl on the Train,Movie,> 2 hrs,6.5,44.0,2021-02-27
65,Geez & Ann,Movie,1-2 hour,8.0,NaN,2021-02-26
67,The Last Bastion,Series,< 30 minutes,8.5,NaN,2021-02-25
71,Red,Movie,> 2 hrs,7.0,72.0,2021-02-23
72,Made You Look: A True Story About Fake Art,Movie,1-2 hour,7.0,NaN,2021-02-23


### Milestone 3: Cleaning the awards data from Wikipedia

In [20]:
# Read wikipedia data

import pandas as pd
import requests
from bs4 import BeautifulSoup

wikiurl="https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)

# find the data table
soup = BeautifulSoup(response.text, 'html.parser')
awards=soup.find('table',{'class':"wikitable"})

df=pd.read_html(str(awards))

# convert to a dataframe
awards_df=pd.DataFrame(df[0])

# Step 1: Clean the Year, Awards, and Nominations columns
# If two Years or numbers are given for a single title (i.e. "2020/21"), keep the first.

awards_df["Year"] = awards_df["Year"].str[:4]

# Find numbers up until the parentheses and brackets
awards_df["Awards"] = awards_df["Awards"].str.extract(r'(\d+)')
awards_df["Nominations"] = awards_df["Nominations"].str.extract(r'(\d+)')

# Step 2: Convert Year, Awards, and Nominations to integer data types

awards_df["Year"] = awards_df["Year"].astype(int)
awards_df["Awards"] = awards_df["Awards"].astype(int)
awards_df["Nominations"] = awards_df["Nominations"].astype(int)

# Step 3: Calculate a win percentage and save in a column called WinPercent

awards_df["WinPercent"] = awards_df["Awards"].astype(int)/awards_df["Nominations"].astype(int)

# Step 4: Create a Date column from the Year column using datetime

from datetime import datetime

awards_df["Date"] = pd.to_datetime(awards_df["Year"], format='%Y')

# Step 5: Remove rows where Year is after 2021

awards_df = awards_df.loc[awards_df["Year"] < 2022]

In [21]:
# Show the first few rows of the awards data set

awards_df.head(5)

,Film,Year,Awards,Nominations,WinPercent,Date
13,CODA,2021,3,3,1.000000,2021-01-01
14,Dune,2021,6,10,0.600000,2021-01-01
15,The Eyes of Tammy Faye,2021,2,2,1.000000,2021-01-01
16,No Time to Die,2021,1,3,0.333333,2021-01-01
17,The Windshield Wiper,2021,1,1,1.000000,2021-01-01


### Milestone 4: Cleaning the API stock data

In [22]:
# Import the data and store it in a data frame

import requests
import pandas as pd

# supress warnings
import warnings
warnings.filterwarnings("ignore")

# The API URL for historical stock prices
URL = 'https://api.stockdata.org/v1/data/eod?'

# declare data range and stock type
params = {'symbols':'NFLX', 
          'date_from':'2020-01-01', 
          'date_to':'2022-06-30', 
          'api_token':'UiCADiIAtUtQJ0K4ZbBBDZotCkvWWuPc6WFzJv92'}

r = requests.get(url = URL, params = params)

# store as a json
r = r.json()

# create the dataframe which will store the stock price data
stock_df = pd.DataFrame(columns=["Date","price_open","price_high",
                        "price_low","price_close","volume"])

# loop through the json data and append the data from the API to stock_df
for i in r['data']:
    date = i['date']
    price_open = i['open']
    price_high = i['high']
    price_low = i['low']
    price_close = i['close']
    volume = i['volume']
    
    stock_df = stock_df.append({'Date':date,
                                'price_open':price_open,
                                'price_high':price_high,
                                'price_low':price_low,
                                'price_close':price_close,
                                'volume':volume},ignore_index=True)

In [23]:
# Step 1: Create a "Year" column from the date column.
# This will be used to join this data to the other two data sources.

stock_df['Year'] = pd.DatetimeIndex(stock_df['Date']).year

# Step 2: Create an "average_price" column
# This will be the price used in the analysis

stock_df['average_price'] = (stock_df['price_high']+stock_df['price_low'])/2

# round this to two decimal places
stock_df['average_price'] = round(stock_df['average_price'],2)

# Step 3: Reformat the date column to year-month-day
# This will be used to join the data with the flat file data by date

# convert date to datetime data type
stock_df['Date'] = pd.to_datetime(stock_df['Date'], errors='coerce').dt.tz_localize(None)
#stock_df['Date'] = stock_df['Date'].dt.date

# Step 4: Add a "stock_symbol" column
# Users will know which stock this dataframe is for

stock_df['stock_symbol'] = 'NFLX'

# Step 5: Remove the "price_high" and "price_low" columns.
# These won't be used in the analysis and will only take up more storage.

stock_df = stock_df.drop(['price_high','price_low'], axis=1)

In [24]:
# Show the final stock data set

stock_df.head(5)

,Date,price_open,price_close,volume,Year,average_price,stock_symbol
0,2020-06-29,445,447.40,110923,2020,440.02,NFLX
1,2020-06-26,465.52,443.07,189062,2020,455.24,NFLX
2,2020-06-25,458.99,466.74,49198,2020,460.52,NFLX
3,2020-06-24,469.26,457.57,93127,2020,463.14,NFLX
4,2020-06-23,466.24,466.30,327781,2020,469.32,NFLX


In [25]:
# Merge the stock data with the Netflix data
# This is done outside of SQLite because SQLite doesn't support datetime data types

stock_netflix_df = pd.merge(netflix_df, stock_df, left_on = 'Netflix Release Date', right_on = 'Date')

stock_netflix_df.head(5)

,Title,Series or Movie,Runtime,IMDb Score,Rotten Tomatoes Score,Netflix Release Date,Date,price_open,price_close,volume,Year,average_price,stock_symbol
0,All For Love,Series,< 30 minutes,6.7,44.0,2020-06-26,2020-06-26,465.52,443.07,189062,2020,455.24,NFLX
1,Home Game,Series,< 30 minutes,7.1,NaN,2020-06-26,2020-06-26,465.52,443.07,189062,2020,455.24,NFLX
2,Eurovision Song Contest: The Story of Fire Saga,Movie,> 2 hrs,6.5,NaN,2020-06-26,2020-06-26,465.52,443.07,189062,2020,455.24,NFLX
3,Ordinary People,Movie,1-2 hour,7.7,89.0,2020-06-26,2020-06-26,465.52,443.07,189062,2020,455.24,NFLX
4,Vivah,Movie,> 2 hrs,6.6,90.0,2020-06-25,2020-06-25,458.99,466.74,49198,2020,460.52,NFLX


### Join the tables

In [26]:
# import sqlite3 and load the tables

import sqlite3

conn = sqlite3.connect(':memory:')

netflix_df.to_sql(name='netflix', con=conn, if_exists='append')
awards_df.to_sql(name='awards', con=conn, if_exists='append')
stock_df.to_sql(name='stock', con=conn, if_exists='append')
stock_netflix_df.to_sql(name='netflix_stock', con=conn, if_exists='append')

257

In [27]:
# Check that all tables are uploaded
pd.read_sql_query("SELECT * FROM netflix LIMIT 5", conn)

,index,Title,Series or Movie,Runtime,IMDb Score,Rotten Tomatoes Score,Netflix Release Date
0,61,The Girl on the Train,Movie,> 2 hrs,6.5,44.0,2021-02-27 00:00:00
1,65,Geez & Ann,Movie,1-2 hour,8.0,NaN,2021-02-26 00:00:00
2,67,The Last Bastion,Series,< 30 minutes,8.5,NaN,2021-02-25 00:00:00
3,71,Red,Movie,> 2 hrs,7.0,72.0,2021-02-23 00:00:00
4,72,Made You Look: A True Story About Fake Art,Movie,1-2 hour,7.0,NaN,2021-02-23 00:00:00


In [28]:
pd.read_sql_query("SELECT * FROM awards LIMIT 5", conn)

,index,Film,Year,Awards,Nominations,WinPercent,Date
0,13,CODA,2021,3,3,1.000000,2021-01-01 00:00:00
1,14,Dune,2021,6,10,0.600000,2021-01-01 00:00:00
2,15,The Eyes of Tammy Faye,2021,2,2,1.000000,2021-01-01 00:00:00
3,16,No Time to Die,2021,1,3,0.333333,2021-01-01 00:00:00
4,17,The Windshield Wiper,2021,1,1,1.000000,2021-01-01 00:00:00


In [29]:
pd.read_sql_query("SELECT * FROM stock LIMIT 5", conn)

,index,Date,price_open,price_close,volume,Year,average_price,stock_symbol
0,0,2020-06-29 00:00:00,445,447.40,110923,2020,440.02,NFLX
1,1,2020-06-26 00:00:00,465.52,443.07,189062,2020,455.24,NFLX
2,2,2020-06-25 00:00:00,458.99,466.74,49198,2020,460.52,NFLX
3,3,2020-06-24 00:00:00,469.26,457.57,93127,2020,463.14,NFLX
4,4,2020-06-23 00:00:00,466.24,466.30,327781,2020,469.32,NFLX


In [32]:
# Join the tables

# LEFT JOIN the netflix data with the stock data on date, and the awards data on title
sql_query = """SELECT *
FROM netflix_stock  
LEFT JOIN awards
on netflix_stock.[Title] = awards.Film
WHERE average_price IS NOT NULL"""

# Executing the query and saving to a dataframe
final_df = pd.read_sql_query(sql_query, conn)

# Closing the connection
conn.close() 

In [38]:
# Check the joined table
final_df.head()

,index,Title,Series or Movie,Runtime,IMDb Score,Rotten Tomatoes Score,Netflix Release Date,Date,price_open,price_close,...,Year,average_price,stock_symbol,index,Film,Year,Awards,Nominations,WinPercent,Date
0,0,All For Love,Series,< 30 minutes,6.7,44.0,2020-06-26 00:00:00,2020-06-26 00:00:00,465.52,443.07,...,2020,455.24,NFLX,NaN,None,NaN,NaN,NaN,NaN,None
1,1,Home Game,Series,< 30 minutes,7.1,NaN,2020-06-26 00:00:00,2020-06-26 00:00:00,465.52,443.07,...,2020,455.24,NFLX,NaN,None,NaN,NaN,NaN,NaN,None
2,2,Eurovision Song Contest: The Story of Fire Saga,Movie,> 2 hrs,6.5,NaN,2020-06-26 00:00:00,2020-06-26 00:00:00,465.52,443.07,...,2020,455.24,NFLX,NaN,None,NaN,NaN,NaN,NaN,None
3,3,Ordinary People,Movie,1-2 hour,7.7,89.0,2020-06-26 00:00:00,2020-06-26 00:00:00,465.52,443.07,...,2020,455.24,NFLX,602.0,Ordinary People,1980.0,4.0,6.0,0.666667,1980-01-01 00:00:00
4,4,Vivah,Movie,> 2 hrs,6.6,90.0,2020-06-25 00:00:00,2020-06-25 00:00:00,458.99,466.74,...,2020,460.52,NFLX,NaN,None,NaN,NaN,NaN,NaN,None


### Visualizations: See PDF of Power BI Visualizations

In [43]:
# Export the dataset to a csv

import csv

final_df.to_csv('final_df.csv')

### Summary

This project proved to be a great learning experience, especially when it comes to handling multiple data sets. I learned a lot about cleaning data, creating a standard key to link data, and creating visuals using Power BI. Once all of the data was pulled from the three data sources, I needed to filter some of the data to only include necessary columns and rows. This ensured that the runtime of my code wasn't too long, and I could focus on the data that would be used in the analysis rather than cleaning data that wouldn't be necessary. Once the data was filtered, I then cleaned the data and added columns that would be used as keys when the data sets were merged. This was perhaps the most challenging aspect for me. I needed to learn how to manipulate datetime columns to be in a consistent format throughout all three data sources in order to join them. Lastly, I was able to practice my Power BI skills by creating visuals that would be meaningful for the analysis.

During this project, I kept in mind some ethical considerations, particularly while cleaning the data. As mentioned above, I filtered the data sources to only include columns and rows which I deemed necessary. This can seem harmless on the surface, but filtering data should be scrutinized since biases can skew the results of the analysis. For example, I could have pulled a specific date range of stock data that would show Netflix's stock price declining if I had wanted to publish a result showing Netflix as a bad investment. Data analysts should not skew the analysis to fit a preconceived result.